<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/RAG_OpenAI/05_question_answering_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r drive/MyDrive/z-發布/"70 單元_2025H1"/"AI 課程發佈"/週三/RAG_OpenAI/docs .

# Question Answering

## Overview

Recall the overall workflow for retrieval augmented generation (RAG):

![overview.jpeg](attachment:overview.jpeg)

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

Let's load our vectorDB.

In [15]:
#!pip install dotenv
#!pip install -U langchain-community
#!pip install chromadb
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00


In [5]:
import os
import openai
import sys
from getpass import getpass
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key  = os.environ['OPENAI_API_KEY']

openai.api_key = getpass("請輸入你的 OpenAI API 金鑰：")

請輸入你的 OpenAI API 金鑰：··········


The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023.
LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [6]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"

print(llm_name)

gpt-3.5-turbo


In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
#embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai.api_key
)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [13]:
print(vectordb._collection.count())

208


In [16]:
# 這裡會出錯，因為 openai 用的 default embedding text 已經不一樣...

question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [17]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key)

<ipython-input-17-c14ad9c02e56>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key)


### RetrievalQA chain

In [18]:
from langchain.chains import RetrievalQA

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [20]:
result = qa_chain({"query": question})

<ipython-input-20-ef85128c39b8>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [21]:
result["result"]

'The major topics covered in this class include linear regression, logistic regression, regularization techniques, neural networks, support vector machines, clustering algorithms, dimensionality reduction, anomaly detection, recommender systems, and large-scale machine learning.'

### Prompt

In [22]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [23]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [24]:
question = "Is probability a class topic?"

In [25]:
result = qa_chain({"query": question})

In [26]:
result["result"]

"I don't know, thanks for asking!"

In [27]:
result["source_documents"][0]

Document(metadata={'total_pages': 18, 'page': 3, 'creator': 'PScript5.dll Version 5.2.2', 'source': './MachineLearning-Lecture02.pdf', 'author': '', 'page_label': '4', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'title': '', 'moddate': '2008-07-11T11:25:05-07:00', 'creationdate': '2008-07-11T11:25:05-07:00'}, page_content="may have more than one input feature. So for example, if instead of just knowing the size \nof the houses, if we know also the number of bedrooms in these houses, let's say, then, so \nif our training set also has a second feature, the number of bedrooms in the house, then \nyou may, let's say X1 denote the size and square feet. Let X have script two denote the \nnumber of bedrooms, and then I would write the hypothesis, H of X, as theta rho plus \ntheta 1X1 plus theta 2X2.  \nOkay, and sometimes when I went to take the hypothesis H, and when I went to make \nthis dependent on the theta is explicit, I'll sometimes write this as H subscript theta of X. \nAnd so t

### RetrievalQA chain types

In [28]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [29]:
result = qa_chain_mr({"query": question})

In [30]:
result["result"]

'Based on the extracted parts of the document provided, probability does not appear to be a class topic. The focus seems to be on regression techniques and machine learning algorithms for predicting house prices based on features.'

If you wish to experiment on the `LangSmith platform` (previously known as LangChain Plus):

 * Go to [LangSmith](https://www.langchain.com/langsmith) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [ ]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..." # replace dots with your api key

In [31]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'Based on the extracted parts of the document provided, probability is not explicitly mentioned as a class topic in the context of locally weighted linear regression, gradient descent, and linear regression for housing prices data. Therefore, it is unclear from the given information whether probability is a class topic in this specific context.'

In [32]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'The discussion is focused on locally weighted linear regression in the context of machine learning, specifically on the topic of gradient descent and convergence to a local minimum. Probability may not be the main topic of discussion in this particular context. However, probability is a fundamental concept in machine learning and is often covered in machine learning courses as it forms the basis for many algorithms and models. In the specific example provided, the algorithm of locally weighted regression is described as a non-parametric learning algorithm that allows for less reliance on carefully choosing features. This algorithm can be used without the need to manually select and adjust features, making it a valuable tool in machine learning applications. If you have specific questions about probability in the context of machine learning or any other topic, feel free to ask!'

### RetrievalQA limitations

QA fails to preserve conversational history.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

Note, The LLM response varies. Some responses **do** include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.